
## Ingest drivers.json file


#### Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), nullable=True),
    StructField("surname", StringType(), nullable=True)
])

In [0]:
drivers_schema = StructType(fields=[
    StructField("driverId", IntegerType(), nullable=False),
    StructField("driverRef", StringType(), nullable=True),
    StructField("number", IntegerType(), nullable=True),
    StructField("code", StringType(), nullable=True),
    StructField("name", name_schema),
    StructField("dob", DateType(), nullable=True),
    StructField("nationality", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True)
])

In [0]:
drivers_df = spark.read.format("json")\
                       .schema(drivers_schema)\
                       .load("/mnt/tideformula1dl/raw/drivers.json")


#### Step 2 - Rename columns and add new comlumns
1. driverId renamed to driver_id
2. driverRef renamed to driver_ref
3. ingestion date added
4. name added with concatenation of forename and surname

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id")\
                                    .withColumnRenamed("driverRef", "driver_ref")\
                                    .withColumn("ingestion_date", current_timestamp())\
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))


#### Step 3 - Drop the unwanted columns
1. name.forename
2. name.surname
3. url

In [0]:
driver_final_df = drivers_with_columns_df.drop(col("url"))


#### Step 4 - Write to the ouput to processed container in parquet format

In [0]:
driver_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [0]:
dbutils.notebook.exit("Success")